In [1]:
#data ; depressed tweet
#Feature Engineering; ' Glove'
#model
#- simple RNN
#- simple LSTM


#evaluating
#- precision
#- recall
#- F-1 score
#- Accuracy

In [1]:
import torch
from torchtext import data

SEED = 1234
import pandas as pd
import numpy as np
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext

import nltk

import random
from sklearn.metrics import classification_report

import pyprind
%matplotlib inline

In [5]:
main_df = pd.read_csv('C:/Users/user/Desktop/mlmd/sentiment_tweets3.csv')
print(main_df.shape)
main_df.head()

(10314, 3)


,Unnamed: 0,message,label
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


In [6]:
main_df = main_df[["message", "label"]]
main_df.head()

,message,label
0,just had a real good moment. i missssssssss hi...,0
1,is reading manga http://plurk.com/p/mzp1e,0
2,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,@lapcat Need to send 'em to my accountant tomo...,0
4,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


In [8]:
main_df.label.value_counts()

0    8000
1    2314
Name: label, dtype: int64

In [14]:
normal_class = main_df.loc[main_df.label == 0, :]
depress_class = main_df.loc[main_df.label == 1, :]

In [15]:
# splitting test and train 
test_normal = normal_class.iloc[:int(0.2*(len(normal_class))), :]
test_depress = depress_class.iloc[:int(0.2*(len(depress_class))), :]

train_normal = normal_class.iloc[int(0.2*(len(normal_class))):, :]
train_depress = depress_class.iloc[int(0.2*(len(depress_class))):, :]

In [16]:
train = pd.concat([train_normal, train_depress], axis=0)
print(train.shape)

test = pd.concat([test_normal, test_depress], axis=0)
print(test.shape)

(8252, 2)
(2062, 2)


In [18]:
train.label.value_counts()

0    6400
1    1852
Name: label, dtype: int64

In [19]:
test.label.value_counts()

0    1600
1     462
Name: label, dtype: int64

In [24]:
train.to_csv("C:/Users/user/Desktop/mlmd/torchtext_data/train.csv", index=False)
test.to_csv("C:/Users/user/Desktop/mlmd/torchtext_data/test.csv", index=False)

In [29]:
!pip install spacy==1.8.2
import spacy
spacy_en = spacy.load('en')

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Stored in directory: C:\Users\user\AppData\Local\pip\Cache\wheels\63\08\c9\4598408c0a93c6a4b008de44bd2924b264d1ed6390c5e17c03
Successfully built ujson
  Stored in directory: C:\Users\user\AppData\Local\pip\Cache\wheels\d7\65\03\f64f9b14300b59cd5b473cf243d5390ee8b4cc25354c41ff0d
  Stored in directory: C:\Users\user\AppData\Local\pip\Cache\wheels\90\af\51\9efd49862c6dcb6439baaa235714fc4de5cecf3e01613b2fef
  Stored in directory: C:\Users\user\AppData\Local\pip\Cache\wheels\1f\36\8b\306d475aa414ff9fcec211da3da5d5e59582219d57f7ea9fa1
  Stored in directory: C:\Users\user\AppData\

In [31]:
is_cuda = torch.cuda.is_available()
print("Cuda Status on system is {}".format(is_cuda))

Cuda Status on system is True


In [32]:
def tokenizer(text):
    return [tok for tok in nltk.word_tokenize(text)]

TEXT = data.Field(sequential=True, tokenize="spacy")
LABEL = data.LabelField(dtype=torch.long, sequential=False)



    Only loading the 'en' tokenizer.



In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [36]:
train_data, test_data = data.TabularDataset.splits(
    path="C:/Users/user/Desktop/mlmd/torchtext_data/", train="train.csv", test="test.csv",format="csv", skip_header=True, 
    fields=[('Text', TEXT), ('Label', LABEL)]
)

In [37]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 8252
Number of testing examples: 2062


In [41]:
TEXT.build_vocab(train_data, vectors=torchtext.vocab.GloVe(name='840B', dim=300), 
                 max_size=20000, min_freq=10)
LABEL.build_vocab(train_data)

.vector_cache\glove.840B.300d.zip: 2.18GB [38:39, 939kB/s]                                                             
100%|█████████████████████████████████████████████████████████████████████▉| 2195471/2196017 [05:32<00:00, 6896.54it/s]

In [42]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 1330
Unique tokens in LABEL vocabulary: 2


In [43]:
TEXT.vocab.freqs.most_common(10)

[('.', 4440),
 ('!', 4408),
 (',', 3406),
 ('I', 3117),
 ('to', 3048),
 (' ', 3010),
 ('the', 2817),
 ('a', 2388),
 ('and', 2084),
 ('you', 1949)]

In [44]:
BATCH_SIZE = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# keep in mind the sort_key option 
train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data), sort_key=lambda x: len(x.Text),
    batch_size=BATCH_SIZE,
    device=device)

In [45]:
LABEL.vocab.freqs

Counter({'0': 6400, '1': 1852})

In [46]:
class simpleRNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):

        #x = [sent len, batch size]
        
        embedded = self.embedding(x)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        out = self.fc(hidden)
        return out

In [47]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 374
OUTPUT_DIM = 2

model1 = simpleRNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([1330, 300])


In [49]:
model1.embedding.weight.data = pretrained_embeddings.cuda()

In [110]:
class_weights = torch.tensor([1.0, 15.0]).cuda()
optimizer = optim.Adam(model1.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [54]:
model1 = model1.to(device)
criterion = criterion.to(device)

In [55]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    preds, ind= torch.max(F.softmax(preds, dim=-1), 1)
    correct = (ind == y).float()
    acc = correct.sum()/float(len(correct))
    return acc


In [111]:
def train(epochs, model, iterator, optimizer, criterion):
    for epoch in range(1,epochs+1):
    
        training_loss = 0.0
        epoch_acc = 0.0

        model.train()
        for batch in iterator:

            optimizer.zero_grad()

            predictions = model(batch.Text).squeeze(0)
        #         print(predictions.shape, batch.Label.shape, model(batch.Text).shape)
            loss = criterion(predictions, batch.Label)
        #         print(loss.shape)
            acc = binary_accuracy(predictions, batch.Label)

            loss.backward()

            optimizer.step()

            training_loss += loss.item()
            epoch_acc += acc.item()
        
        training_loss /= len(iterator)
        epoch_acc /= len(iterator)
        print(f'Epoch: {epoch+1:02}, Train Loss: {training_loss:.3f}, Train Acc: {epoch_acc*100:.2f}% ')

In [112]:
train(10,model1,train_iterator, optimizer, criterion)

Epoch: 02, Train Loss: 0.561, Train Acc: 28.08% 
Epoch: 03, Train Loss: 0.516, Train Acc: 22.85% 
Epoch: 04, Train Loss: 0.514, Train Acc: 24.24% 
Epoch: 05, Train Loss: 0.512, Train Acc: 22.92% 
Epoch: 06, Train Loss: 0.507, Train Acc: 24.43% 
Epoch: 07, Train Loss: 0.508, Train Acc: 23.28% 
Epoch: 08, Train Loss: 0.508, Train Acc: 22.85% 
Epoch: 09, Train Loss: 0.513, Train Acc: 23.07% 
Epoch: 10, Train Loss: 0.501, Train Acc: 22.60% 
Epoch: 11, Train Loss: 0.517, Train Acc: 24.27% 


In [84]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        bar = pyprind.ProgBar(len(iterator), bar_char='█')
        for batch in iterator:

            predictions = model(batch.Text).squeeze(0)
            
            loss = criterion(predictions, batch.Label)
            
            acc = binary_accuracy(predictions, batch.Label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            bar.update()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


test_loss, test_acc = evaluate(model1, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')

0% [██████████████████████████████] 100% | ETA: 00:00:00

| Test Loss: 1.840 | Test Acc: 23.08% |



Total time elapsed: 00:00:00


In [85]:
def predict_sentiment(sentence,model):
    tokenized = [tok for tok in sentence.split()]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    
    tensor = tensor.unsqueeze(1)
#     print(tensor.shape)
    prediction = model(tensor)
#     print(prediction)
    preds, ind= torch.max(F.softmax(prediction.squeeze(0), dim=-1), 1)
#     print(preds)
    return preds, ind

In [87]:
test = pd.read_csv("C:/Users/user/Desktop/mlmd/torchtext_data/test.csv")
pre = [predict_sentiment(k,model1)[1].item() for k in test.message]
print(classification_report(test.label, pre))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1600
           1       0.22      1.00      0.37       462

    accuracy                           0.22      2062
   macro avg       0.11      0.50      0.18      2062
weighted avg       0.05      0.22      0.08      2062



C:\Users\user\Anaconda3\envs\ML\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [97]:
class lstmRNN(nn.Module):
    def __init__(self, hidden_size, embedding_dim, vocab_size):
        super(lstmRNN, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(input_size = embedding_dim,hidden_size = hidden_size, num_layers = 1)
        self.fc = nn.Linear(hidden_size, 2)
        
    def forward(self, x):

        #x = [sent len, batch size]
        
        embedded = self.embedding(x)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, (hidden,_) = self.encoder(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        preds = self.fc(hidden.squeeze(0))
        return preds

In [102]:
model2 = lstmRNN(HIDDEN_DIM,EMBEDDING_DIM,INPUT_DIM)
model2 = model2.to(device)
optimizer = optim.Adam(model2.parameters(), lr=1e-3)

def train2(epochs, model, iterator, optimizer, criterion):
    for epoch in range(1,epochs+1):
    
        training_loss = 0.0
        epoch_acc = 0.0

        model.train()
        for batch_idx, batch in enumerate(iterator):

            optimizer.zero_grad()

            predictions = model(batch.Text)
        #         print(predictions.shape, batch.Label.shape, model(batch.Text).shape)
            loss = criterion(predictions, batch.Label)
        #         print(loss.shape)
            acc = binary_accuracy(predictions, batch.Label)

            loss.backward()

            optimizer.step()

            training_loss += loss.data.item()*batch.Text.size(0)
            epoch_acc += acc.item()
        
        training_loss /= len(iterator)
        epoch_acc /= len(iterator)
        print(f'Epoch: {epoch+1:02}, Train Loss: {training_loss:.3f}, Train Acc: {epoch_acc*100:.2f}% ')

In [103]:
train2(100, model2, train_iterator, optimizer, criterion)

Epoch: 02, Train Loss: 13.154, Train Acc: 66.39% 
Epoch: 03, Train Loss: 2.571, Train Acc: 98.07% 
Epoch: 04, Train Loss: 1.424, Train Acc: 99.04% 
Epoch: 05, Train Loss: 0.571, Train Acc: 99.43% 
Epoch: 06, Train Loss: 0.379, Train Acc: 99.62% 
Epoch: 07, Train Loss: 0.208, Train Acc: 99.72% 
Epoch: 08, Train Loss: 0.129, Train Acc: 99.81% 
Epoch: 09, Train Loss: 0.116, Train Acc: 99.88% 
Epoch: 10, Train Loss: 0.087, Train Acc: 99.87% 
Epoch: 11, Train Loss: 0.087, Train Acc: 99.89% 
Epoch: 12, Train Loss: 0.228, Train Acc: 99.85% 
Epoch: 13, Train Loss: 0.586, Train Acc: 99.33% 
Epoch: 14, Train Loss: 0.585, Train Acc: 99.47% 
Epoch: 15, Train Loss: 0.139, Train Acc: 99.85% 
Epoch: 16, Train Loss: 0.241, Train Acc: 99.61% 
Epoch: 17, Train Loss: 0.147, Train Acc: 99.85% 
Epoch: 18, Train Loss: 0.067, Train Acc: 99.93% 
Epoch: 19, Train Loss: 0.055, Train Acc: 99.95% 
Epoch: 20, Train Loss: 0.066, Train Acc: 99.95% 
Epoch: 21, Train Loss: 0.053, Train Acc: 99.96% 
Epoch: 22, Train Lo

In [105]:
test_loss, test_acc = evaluate(model2, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')

0% [██████████████████████████████] 100% | ETA: 00:00:00

| Test Loss: 0.084 | Test Acc: 99.57% |



Total time elapsed: 00:00:00


In [122]:
def predict_sentiment(sentence,model):
    tokenized = [tok for tok in sentence.split()]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    
    tensor = tensor.unsqueeze(1)
#     print(tensor.shape)
    prediction = model(tensor)
#     print(prediction)
    preds, ind= torch.max(F.softmax(prediction, dim=-1), 1)
#     print(preds)
    return preds, ind

pre = [predict_sentiment(k,model2)[1].item() for k in test.message]

print(classification_report(test.label, pre))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96      1600
           1       0.99      0.73      0.84       462

    accuracy                           0.94      2062
   macro avg       0.96      0.86      0.90      2062
weighted avg       0.94      0.94      0.93      2062



In [ ]:
#ULMFiT
